In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.keras
import tensorflow.keras.datasets.mnist as mnist_keras
from tensorflow.examples.tutorials.mnist import input_data

# 查看数据集

In [ ]:
# tensorflow
mnist = input_data.read_data_sets("../datasets/MNIST_qata", one_hot=True)

train_images = mnist.train.images
train_labels = mnist.train.labels
test_images = mnist.test.images
test_labels = mnist.test.labels

train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

# 使用神经网络（3层）

In [ ]:
# 初始化权重函数
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

# 定义模型函数(线性的模型)
def model(X, w_1, b_1, w_2, b_2, w_o, b_o):
    layer = tf.matmul(X, w_1) + b_1
    layer = tf.nn.relu(layer)
    
    layer = tf.matmul(layer, w_2) + b_2
    layer = tf.nn.relu(layer)
    
    layer = tf.matmul(layer, w_o) + b_o
    h = tf.nn.sigmoid(layer)
    return h

# 获取数据
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

# 定义占位符
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

layer1_size = 625
layer2_size = 500

# 初始化权重
w_1 = init_weights([784, layer1_size]) # create symbolic variables
b_1 = tf.Variable(tf.random_normal([layer1_size], stddev=0.01))

w_2 = init_weights([layer1_size, layer2_size])
b_2 = tf.Variable(tf.random_normal([layer2_size], stddev=0.01))

w_o = init_weights([layer2_size, 10])
b_o = tf.Variable(tf.random_normal([10], stddev=0.01))

# 进行模型运算得出每个样本的10个结果
py_x = model(X, w_1, b_1, w_2, b_2, w_o, b_o)
print(py_x)

# 损失值
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y)) # compute costs

# 梯度下降
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) # construct an optimizer

# 获得10个结果里最大值的索引，即预测值（可能是0-9中的一个）的索引
predict_op = tf.argmax(py_x, 1)

In [ ]:
# 开启会话，开始训练
with tf.Session() as sess:
    # 初始化所有变量
    tf.global_variables_initializer().run()

    # 开始训练，训练10轮
    for i in range(300):
        '''
        每个循环训练一个batch，从[0,128]开始到[128,255]...
        ([0, 128, 256, 384, 512, 640, 768, 896, 1024, 1152],
         [128, 256, 384, 512, 640, 768, 896, 1024, 1152, 1280])
        '''
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
        
        # 准确度，但是这是线性的net，
        print(i, np.mean(np.argmax(teY, axis=1) ==sess.run(predict_op, feed_dict={X: teX})))
        #该轮 测试集的真实值 是否等于 训练结果即预测值
    print(i, np.mean(np.argmax(trY, axis=1) ==sess.run(predict_op, feed_dict={X: trX})))